In [497]:
import pandas as pd
from tabulate import tabulate

In [498]:
#pd.set_option('display.max_colwidth', None)

In [494]:
def process_multiple_option_column(df, column_name):
    question_sts = df[column_name].to_frame()

    # Criando um dicionário para armazenar as contagens
    answers_count = {}
    for row in question_sts.values:
        answers_selected = row[0].split(';')
        for answer in answers_selected:
            answer = answer.strip()
            if answer in answers_count:
                answers_count[answer] += 1
            else:
                answers_count[answer] = 1

    # Convertendo o dicionário em um DataFrame
    df_answers = pd.DataFrame.from_dict(answers_count, orient='index', columns=['qtde'])
    df_answers.index.names = ['answer']  # Renomeando o índice para 'answer'
    df_answers = df_answers.reset_index()  # Convertendo o índice em coluna

    df_answers['pct'] = ((df_answers['qtde'] / df.shape[0]) * 100).round(1)

    return df_answers

In [495]:
def process_level_knowledge_column(df, column_name):
    question_sts = df[column_name].value_counts().to_frame()

    question_sts = question_sts.reset_index(drop=False)
    question_sts = question_sts.rename(columns={'index': 'question_anwser'})
    question_sts.rename(columns={column_name: 'qtde'}, inplace=True)

    question_sts["question_anwser"] = question_sts["question_anwser"].str.split('(', expand=True)[0].str.strip()

    # Criando um dicionário para mapear os níveis de conhecimento para valores numéricos
    dicionario_mapeamento = {'Sem conhecimento': 1,
                             'Noções básicas': 2,
                             'Conhecimento básico': 3,
                             'Conhecimento bom': 4,
                             'Conhecimento completo': 5}

    # Criando a nova coluna 'ord_question_anwser'
    question_sts['ord_question_anwser'] = question_sts['question_anwser'].map(dicionario_mapeamento)

    question_sts = question_sts.sort_values(by='ord_question_anwser', ascending=True)
    question_sts.drop('ord_question_anwser', axis=1, inplace=True)
    question_sts = question_sts.reset_index(drop=True)

    question_sts['pct'] = ((question_sts['qtde'] / df.shape[0]) * 100).round(1)

    # Dicionário de tradução
    translation_dict = {
        'Sem conhecimento': 'No knowledge',
        'Noções básicas': 'Basic Awareness',
        'Conhecimento básico': 'Basic Knowledge',
        'Conhecimento bom': 'Intermediate Knowledge',
        'Conhecimento completo': 'Comprehensive Knowledge'
    }

    # Atualizando a coluna com os valores traduzidos
    question_sts['question_anwser'] = question_sts['question_anwser'].map(translation_dict)

    return question_sts

In [190]:
df = pd.read_csv('QUESTIONÁRIO ONLINE: Investigando a Percepção dos Desenvolvedores Brasileiros de Software  sobre Privacidade de Dados no uso de  LLMs.csv')

In [191]:
df.shape

(73, 44)

In [192]:
df.columns

Index(['Carimbo de data/hora', 'Você concorda em participar desta pesquisa?',
       '1. Qual a sua idade?', '2. Em qual Estado você mora?',
       '3. Qual seu nível de escolaridade?',
       '4. Qual é a sua principal função atualmente na área de desenvolvimento de software?',
       '5. Quantos anos de experiência você tem trabalhando em desenvolvimento de software?',
       '6. Qual seu nível de conhecimento sobre privacidade de dados?',
       '7. Você já participou de treinamentos ou cursos sobre privacidade de dados? ',
       '8. Qual sua principal fonte de informação sobre privacidade de dados? ',
       '9.  Qual seu nível de conhecimento sobre a Lei Geral de Proteção de Dados (LGPD)? ',
       '10. Qual seu nível de conhecimento sobre os princípios da Lei Geral de Proteção de Dados (LGPD)? ',
       '11. Quais são os principais princípios da LGPD que você conhece? (Marque todas as opções que se aplicam)',
       '12. Quais dos seguintes conceitos de proteção de dados da LGPD

In [359]:
idade_sts = df['1. Qual a sua idade?'].value_counts().to_frame()
idade_sts = idade_sts.sort_index(ascending=True)
idade_sts = idade_sts.reset_index(drop=False)
idade_sts = idade_sts.rename(columns={'index': 'FaixaEtaria'})

# Criando um dicionário para mapear as regioes para valores numéricos
dicionario_mapeamento = {'Entre 26 e 30 anos': 1,
                         'Entre 31 e 36 anos': 2,
                         'Entre 37 e 42 anos': 3,
                         'Entre 43 e 47 anos': 4,
                         "Entre 48 e 54 anos":5,
                         "Entre 55 a 60 anos":6}
# Criando uma nova coluna chamada 'ord_regiao' para posterior ordenacao por ela
idade_sts['ord_faixa_etaria'] = idade_sts['FaixaEtaria'].map(dicionario_mapeamento)

idade_sts = idade_sts.sort_values(by='ord_faixa_etaria', ascending=True)
idade_sts.drop('ord_faixa_etaria', axis=1, inplace=True)
idade_sts = idade_sts.reset_index(drop=True)


idade_sts.rename(columns={'1. Qual a sua idade?': 'qtde'}, inplace=True)
idade_sts['pct'] = ((idade_sts['qtde']/df.shape[0])*100).round(1)

idade_sts['FaixaEtaria'] = idade_sts['FaixaEtaria'].replace(["Entre 26 e 30 anos", 
                                                                         "Entre 31 e 36 anos",
                                                                         "Entre 37 e 42 anos",
                                                                         "Entre 43 e 47 anos",
                                                                         "Entre 48 e 54 anos",
                                                                         "Entre 55 a 60 anos"],
                                                                         ["26 - 30",
                                                                          "31 - 36",
                                                                          "37 - 42",
                                                                          "43 - 47",
                                                                          "48 - 54",
                                                                          "55 - 30"])
idade_sts

,FaixaEtaria,qtde,pct
0,26 - 30,4,5.5
1,31 - 36,11,15.1
2,37 - 42,23,31.5
3,43 - 47,20,27.4
4,48 - 54,12,16.4
5,55 - 30,3,4.1


In [360]:
# Criando a tabela LaTeX com o pacote `tabulate`
tabela_latex = tabulate(idade_sts, floatfmt=".1f", tablefmt="latex", showindex="False")

# Imprimindo a tabela LaTeX
print(tabela_latex)

\begin{tabular}{lrr}
\hline
 26 - 30 &  4 &  5.5 \\
 31 - 36 & 11 & 15.1 \\
 37 - 42 & 23 & 31.5 \\
 43 - 47 & 20 & 27.4 \\
 48 - 54 & 12 & 16.4 \\
 55 - 30 &  3 &  4.1 \\
\hline
\end{tabular}


In [255]:
estados_regiao = {
    "AC": "Norte",
    "AL": "Nordeste",
    "AM": "Norte",
    "AP": "Norte",
    "BA": "Nordeste",
    "CE": "Nordeste",
    "DF": "Centro-Oeste",
    "ES": "Sudeste",
    "GO": "Centro-Oeste",
    "MA": "Nordeste",
    "MG": "Sudeste",
    "MS": "Centro-Oeste",
    "MT": "Centro-Oeste",
    "PA": "Norte",
    "PB": "Nordeste",
    "PE": "Nordeste",
    "PI": "Nordeste",
    "PR": "Sul",
    "RJ": "Sudeste",
    "RN": "Nordeste",
    "RS": "Sul",
    "RO": "Norte",
    "RR": "Norte",
    "SC": "Sul",
    "SE": "Nordeste",
    "SP": "Sudeste",
    "TO": "Norte",
    "Nenhum (moro fora do Brasil)":"Exterior"
}

In [355]:
estado_moradia = df["2. Em qual Estado você mora?"].to_frame()
estado_moradia["Região"] = estado_moradia['2. Em qual Estado você mora?'].map(estados_regiao)
estado_moradia_sts = estado_moradia["Região"].value_counts().to_frame()

estado_moradia_sts = estado_moradia_sts.reset_index(drop=False)
estado_moradia_sts = estado_moradia_sts.rename(columns={'index': 'regiao_brasil','Região':'qtde'})

estado_moradia_sts['pct'] = ((estado_moradia_sts['qtde']/df.shape[0])*100).round(1)

# Criando um dicionário para mapear as regioes para valores numéricos
dicionario_mapeamento = {'Norte': 1,
                         'Nordeste': 2,
                         'Centro-Oeste': 3,
                         'Sudeste': 4,
                         "Sul":5,
                         "Exterior":6}
# Criando uma nova coluna chamada 'ord_regiao' para posterior ordenacao por ela
estado_moradia_sts['ord_regiao'] = estado_moradia_sts['regiao_brasil'].map(dicionario_mapeamento)

estado_moradia_sts = estado_moradia_sts.sort_values(by='ord_regiao', ascending=True)
estado_moradia_sts.drop('ord_regiao', axis=1, inplace=True)
estado_moradia_sts = estado_moradia_sts.reset_index(drop=True)


estado_moradia_sts['regiao_brasil'] = estado_moradia_sts['regiao_brasil'].replace(["Centro-Oeste", 
                                                                         "Exterior",
                                                                         "Nordeste",
                                                                         "Norte",
                                                                         "Sudeste",
                                                                         "Sul"],
                                                                         ["Midwest",
                                                                          "Abroad",
                                                                          "Northeast",
                                                                          "North",
                                                                          "Southeast",
                                                                          "South"])
estado_moradia_sts

,regiao_brasil,qtde,pct
0,North,3,4.1
1,Northeast,1,1.4
2,Midwest,59,80.8
3,Southeast,6,8.2
4,South,3,4.1
5,Abroad,1,1.4


In [356]:
# Criando a tabela LaTeX com o pacote `tabulate`
tabela_latex = tabulate(estado_moradia_sts, floatfmt=".1f", tablefmt="latex", showindex="False")

# Imprimindo a tabela LaTeX
print(tabela_latex)

\begin{tabular}{lrr}
\hline
 North     &  3 &  4.1 \\
 Northeast &  1 &  1.4 \\
 Midwest   & 59 & 80.8 \\
 Southeast &  6 &  8.2 \\
 South     &  3 &  4.1 \\
 Abroad    &  1 &  1.4 \\
\hline
\end{tabular}


In [337]:
nivel_escolaridade_sts = df["3. Qual seu nível de escolaridade?"].value_counts().to_frame()
nivel_escolaridade_sts = nivel_escolaridade_sts.reset_index(drop=False)
nivel_escolaridade_sts = nivel_escolaridade_sts.rename(columns={'index': 'Escolaridade'})

# Criando um dicionário para mapear os níveis de escolaridade para valores numéricos
dicionario_mapeamento = {'Graduado': 1,
                         'Pós-graduação Lato Sensu (Especialização) ou MBA': 2,
                         'Mestrado': 3,
                         'Doutorado': 4}
# Criando uma nova coluna chamada 'ord_escolaridade' para posterior ordenacao por ela
nivel_escolaridade_sts['ord_escolaridade'] = nivel_escolaridade_sts['Escolaridade'].map(dicionario_mapeamento)


nivel_escolaridade_sts = nivel_escolaridade_sts.sort_values(by='ord_escolaridade', ascending=True)
nivel_escolaridade_sts.drop('ord_escolaridade', axis=1, inplace=True)
nivel_escolaridade_sts = nivel_escolaridade_sts.reset_index(drop=True)


nivel_escolaridade_sts.rename(columns={'3. Qual seu nível de escolaridade?': 'qtde'}, inplace=True)
nivel_escolaridade_sts['pct'] = ((nivel_escolaridade_sts['qtde']/df.shape[0])*100).round(1)

nivel_escolaridade_sts['Escolaridade'] = nivel_escolaridade_sts['Escolaridade'].replace(["Graduado", 
                                                                         "Pós-graduação Lato Sensu (Especialização) ou MBA",
                                                                         "Mestrado",
                                                                         "Doutorado"],
                                                                         ["Graduated",
                                                                          "Specialist",
                                                                          "Master",
                                                                          "PhD"])
nivel_escolaridade_sts

,Escolaridade,qtde,pct
0,Graduated,14,19.2
1,Specialist,42,57.5
2,Master,10,13.7
3,PhD,6,8.2


In [338]:
# Criando a tabela LaTeX com o pacote `tabulate`
tabela_latex = tabulate(nivel_escolaridade_sts, floatfmt=".1f", tablefmt="latex", showindex="False")

# Imprimindo a tabela LaTeX
print(tabela_latex)

\begin{tabular}{lrr}
\hline
 Graduated  & 14 & 19.2 \\
 Specialist & 42 & 57.5 \\
 Master     & 10 & 13.7 \\
 PhD        &  6 &  8.2 \\
\hline
\end{tabular}


In [363]:
perfil = df["4. Qual é a sua principal função atualmente na área de desenvolvimento de software?"].replace(["Teste", "AppSec","Administrador de Banco de Dados"], ["Analista de qualidade e/ou testador", "Analista de Segurança de TI","Administrador de Dados"])
perfil_sts = perfil.value_counts().to_frame()

perfil_sts = perfil_sts.reset_index(drop=False)
perfil_sts = perfil_sts.rename(columns={'index': 'perfil'})
perfil_sts.rename(columns={'4. Qual é a sua principal função atualmente na área de desenvolvimento de software?': 'qtde'}, inplace=True)

perfil_sts['pct'] = ((perfil_sts['qtde']/df.shape[0])*100).round(1)

perfil_sts['perfil'] = perfil_sts['perfil'].replace(["Administrador de Dados",
                                                                "Analista de Segurança de TI",
                                                                "Analista de qualidade e/ou testador",
                                                                "Analista de requisitos",
                                                                "Arquiteto de Software",
                                                                "Dono do Produto (Product Owner)",
                                                                "Engenheiro DevOps",
                                                                "Engenheiro e/ou analista de dados",
                                                                "Gerente de projeto",
                                                                "Líder técnico e/ou de equipe",
                                                                "Programador e/ou desenvolvedor"],
                                                                ["Data Administrator",
                                                                "IT Security Analyst",
                                                                "Quality Analyst and/or Tester",
                                                                "Software Requirements Analyst",
                                                                "Software Architect",
                                                                "Product Owner",
                                                                "DevOps Engineer",
                                                                "Data Engineer and/or Analyst",
                                                                "Project Manager",
                                                                "Technical and/or Team Leader",
                                                                "Programmer and/or Developer"])

perfil_sts = perfil_sts.sort_values(by='perfil', ascending=True)
perfil_sts = perfil_sts.reset_index(drop=True)

perfil_sts

,perfil,qtde,pct
0,Data Administrator,1,1.4
1,Data Engineer and/or Analyst,6,8.2
2,DevOps Engineer,5,6.8
3,IT Security Analyst,2,2.7
4,Product Owner,2,2.7
5,Programmer and/or Developer,20,27.4
6,Project Manager,9,12.3
7,Quality Analyst and/or Tester,6,8.2
8,Scrum Master,3,4.1
9,Software Architect,7,9.6


In [364]:
# Criando a tabela LaTeX com o pacote `tabulate`
tabela_latex = tabulate(perfil_sts, floatfmt=".1f", tablefmt="latex", showindex="False")

# Imprimindo a tabela LaTeX
print(tabela_latex)

\begin{tabular}{lrr}
\hline
 Data Administrator            &  1 &  1.4 \\
 Data Engineer and/or Analyst  &  6 &  8.2 \\
 DevOps Engineer               &  5 &  6.8 \\
 IT Security Analyst           &  2 &  2.7 \\
 Product Owner                 &  2 &  2.7 \\
 Programmer and/or Developer   & 20 & 27.4 \\
 Project Manager               &  9 & 12.3 \\
 Quality Analyst and/or Tester &  6 &  8.2 \\
 Scrum Master                  &  3 &  4.1 \\
 Software Architect            &  7 &  9.6 \\
 Software Requirements Analyst &  2 &  2.7 \\
 Technical and/or Team Leader  & 10 & 13.7 \\
\hline
\end{tabular}


In [366]:
experiencia_sts = df["5. Quantos anos de experiência você tem trabalhando em desenvolvimento de software?"].value_counts().to_frame()

experiencia_sts = experiencia_sts.reset_index(drop=False)
experiencia_sts = experiencia_sts.rename(columns={'index': 'experiencia'})
experiencia_sts.rename(columns={'5. Quantos anos de experiência você tem trabalhando em desenvolvimento de software?': 'qtde'}, inplace=True)


# Criando um dicionário para mapear os níveis de escolaridade para valores numéricos
dicionario_mapeamento = {'Menos de 1 ano': 1,
                         'Entre 1 e 3 anos': 2,
                         'Entre 4 e 6 anos': 3,
                         'Entre 7 e 9 anos': 4,
                         'Entre 10 e 12 anos':5,
                         'Entre 13 e 15 anos':6,
                         'Mais de 15 anos':7
                         }
# Criando uma nova coluna chamada 'ord_experiencia' para posterior ordenacao por ela
experiencia_sts['ord_experiencia'] = experiencia_sts['experiencia'].map(dicionario_mapeamento)


experiencia_sts = experiencia_sts.sort_values(by='ord_experiencia', ascending=True)
experiencia_sts.drop('ord_experiencia', axis=1, inplace=True)
experiencia_sts = experiencia_sts.reset_index(drop=True)

experiencia_sts['pct'] = ((experiencia_sts['qtde']/df.shape[0])*100).round(1)

experiencia_sts['experiencia'] = experiencia_sts['experiencia'].replace(["Menos de 1 ano", 
                                                                         "Entre 1 e 3 anos",
                                                                         "Entre 4 e 6 anos",
                                                                         "Entre 7 e 9 anos",
                                                                         "Entre 10 e 12 anos",
                                                                         "Entre 13 e 15 anos",
                                                                         "Mais de 15 anos"],
                                                                         ["< 1",
                                                                          "1 - 3",
                                                                          "4 - 6",
                                                                          "7 - 9",
                                                                          "10 - 12",
                                                                          "13 - 15",
                                                                          "> 15"])

experiencia_sts

,experiencia,qtde,pct
0,< 1,2,2.7
1,1 - 3,5,6.8
2,4 - 6,5,6.8
3,7 - 9,6,8.2
4,10 - 12,10,13.7
5,13 - 15,6,8.2
6,> 15,39,53.4


In [350]:
# Criando a tabela LaTeX com o pacote `tabulate`
tabela_latex = tabulate(experiencia_sts, floatfmt=".1f", tablefmt="latex", showindex="False")

# Imprimindo a tabela LaTeX
print(tabela_latex)

\begin{tabular}{lrr}
\hline
 \ensuremath{<} 1     &  2 &  2.7 \\
 1 - 3   &  5 &  6.8 \\
 4 - 6   &  5 &  6.8 \\
 7 - 9   &  6 &  8.2 \\
 10 - 12 & 10 & 13.7 \\
 13 - 15 &  6 &  8.2 \\
 \ensuremath{>} 15    & 39 & 53.4 \\
\hline
\end{tabular}


In [455]:
result_q06 = process_level_knowledge_column(df, "6. Qual seu nível de conhecimento sobre privacidade de dados?")
result_q06

,question_anwser,qtde,pct
0,Basic Awareness,6,8.2
1,Basic Knowledge,38,52.1
2,Intermediate Knowledge,22,30.1
3,Comprehensive Knowledge,7,9.6


In [458]:
capacitacao_sts = df["7. Você já participou de treinamentos ou cursos sobre privacidade de dados? "].value_counts().to_frame()
capacitacao_sts = capacitacao_sts.reset_index(drop=False)
capacitacao_sts = capacitacao_sts.rename(columns={'index': 'capacitacao_privacidade'})
capacitacao_sts.rename(columns={'7. Você já participou de treinamentos ou cursos sobre privacidade de dados? ': 'qtde'}, inplace=True)

# Dicionário de tradução
translation_dict = {
    'Nunca participei': 'Never',
    'Participei de um treinamento introdutório': 'Introductory training',
    'Participei de um curso básico': 'Basic course',
    'Participei de um curso intermediário': 'Intermediate course',
    'Participei de um treinamento avançado ou especialização': 'Advanced training or specialization'
}

# Atualizando a coluna com os valores traduzidos
capacitacao_sts['capacitacao_privacidade'] = capacitacao_sts['capacitacao_privacidade'].map(translation_dict)



# Criando um dicionário para mapear os níveis de conhecimento para valores numéricos
dicionario_mapeamento = {'Never': 1,
                         'Introductory training': 2,
                         'Basic course': 3,
                         'Intermediate course': 4,
                         'Advanced training or specialization':5}

# Criando a nova coluna 'ord_conhecimento'
capacitacao_sts['ord_capacitacao'] = capacitacao_sts['capacitacao_privacidade'].map(dicionario_mapeamento)

capacitacao_sts = capacitacao_sts.sort_values(by='ord_capacitacao', ascending=True)
capacitacao_sts.drop('ord_capacitacao', axis=1, inplace=True)
capacitacao_sts = capacitacao_sts.reset_index(drop=True)

capacitacao_sts['pct'] = ((capacitacao_sts['qtde']/df.shape[0])*100).round(1)

capacitacao_sts

,capacitacao_privacidade,qtde,pct
0,Never,18,24.7
1,Introductory training,14,19.2
2,Basic course,19,26.0
3,Intermediate course,15,20.5
4,Advanced training or specialization,7,9.6


In [447]:
fonteinformacao_sts = df["8. Qual sua principal fonte de informação sobre privacidade de dados? "].value_counts().to_frame()
fonteinformacao_sts = fonteinformacao_sts.reset_index(drop=False)
fonteinformacao_sts = fonteinformacao_sts.rename(columns={'index': 'fonte_informacao'})
fonteinformacao_sts.rename(columns={'8. Qual sua principal fonte de informação sobre privacidade de dados? ': 'qtde'}, inplace=True)

# Dicionário de tradução
translation_dict = {
    'Internet':'Internet',
    'Cursos': 'Courses',
    'Contato com especialistas': 'Contact with experts',
    'Revistas e artigos': 'Magazines and articles',
    'Organizações e associações': 'Organizations and associations',
    'Eventos e workshops': 'Advanced training or specialization',
    'Atualizações legislativas':'Events and workshops',
    'Várias fontes: portal da ANPD, autoridades de outros países, leis, livros e eventos.':'Multiple sources',
    'Livros':'Books'
}
# Atualizando a coluna com os valores traduzidos
fonteinformacao_sts['fonte_informacao'] = fonteinformacao_sts['fonte_informacao'].map(translation_dict)

fonteinformacao_sts = fonteinformacao_sts.sort_values(by='fonte_informacao', ascending=True)
fonteinformacao_sts = fonteinformacao_sts.reset_index(drop=True)
fonteinformacao_sts['pct'] = ((fonteinformacao_sts['qtde']/df.shape[0])*100).round(1)

fonteinformacao_sts

,fonte_informacao,qtde,pct
0,Advanced training or specialization,3,4.1
1,Books,1,1.4
2,Contact with experts,5,6.8
3,Courses,14,19.2
4,Events and workshops,2,2.7
5,Internet,40,54.8
6,Magazines and articles,4,5.5
7,Multiple sources,1,1.4
8,Organizations and associations,3,4.1


In [456]:
result_q09 = process_level_knowledge_column(df, "9.  Qual seu nível de conhecimento sobre a Lei Geral de Proteção de Dados (LGPD)? ")
result_q09

,question_anwser,qtde,pct
0,No knowledge,1,1.4
1,Basic Awareness,21,28.8
2,Basic Knowledge,31,42.5
3,Intermediate Knowledge,17,23.3
4,Comprehensive Knowledge,3,4.1


In [457]:
result_q10 = process_level_knowledge_column(df, "10. Qual seu nível de conhecimento sobre os princípios da Lei Geral de Proteção de Dados (LGPD)? ")
result_q10

,question_anwser,qtde,pct
0,No knowledge,2,2.7
1,Basic Awareness,24,32.9
2,Basic Knowledge,28,38.4
3,Intermediate Knowledge,16,21.9
4,Comprehensive Knowledge,3,4.1


In [499]:
column_name = "11. Quais são os principais princípios da LGPD que você conhece? (Marque todas as opções que se aplicam)"
result_q11 = process_multiple_option_column(df, column_name)
result_q11

,answer,qtde,pct
0,Finalidade,51,69.9
1,Adequação,33,45.2
2,Necessidade,36,49.3
3,Livre Acesso,32,43.8
4,Qualidade dos Dados,34,46.6
5,Transparência,43,58.9
6,Segurança,55,75.3
7,Prevenção,33,45.2
8,Não Descriminação,26,35.6
9,Responsabilização e prestação de contas,32,43.8


In [500]:
column_name = "12. Quais dos seguintes conceitos de proteção de dados da LGPD você está familiarizado? (Marque todas as opções que se aplicam)"
result_q12 = process_multiple_option_column(df, column_name)
result_q12

,answer,qtde,pct
0,Dados pessoais,64,87.7
1,Dados pessoais sensíveis,63,86.3
2,Tratamento de dados pessoais,48,65.8
3,Tratamento de dados pessoais sensíveis,49,67.1
4,Anonimização e/ou pseudonimização de dados pessoais,43,58.9
5,Agentes de tratamento de dados pessoais,31,42.5
6,Consentimento do titular dos dados,48,65.8
7,"Direitos do titular dos dados (ex: acesso, correção, exclusão)",42,57.5
8,Responsabilização e ressarcimento de danos,25,34.2
9,Sanções por descumprimento da LGPD,22,30.1


In [461]:
result_q13 = process_level_knowledge_column(df, "13. Qual o seu nível de conhecimento sobre as diferenças entre dados pessoais e dados sensíveis? ")
result_q13

,question_anwser,qtde,pct
0,No knowledge,3,4.1
1,Basic Awareness,14,19.2
2,Basic Knowledge,18,24.7
3,Intermediate Knowledge,32,43.8
4,Comprehensive Knowledge,6,8.2


In [462]:
result_q14 = process_level_knowledge_column(df, "14. Qual o seu nível de conhecimento sobre os direitos dos titulares de dados sob a LGPD?")
result_q14

,question_anwser,qtde,pct
0,No knowledge,7,9.6
1,Basic Awareness,17,23.3
2,Basic Knowledge,28,38.4
3,Intermediate Knowledge,17,23.3
4,Comprehensive Knowledge,4,5.5
